In [ ]:
%matplotlib inline
from importlib import reload

In [ ]:
import re, time, os.path as path
from datetime import date
from math import floor
from concurrent.futures import ThreadPoolExecutor, as_completed

import my_types, count

start = time.perf_counter()

max_workers = 15
root_dir = "/home/bwhite/Projects/bryanchriswhite"
repo_names = [
    "storj",
    "uplink",
    "uplink-c"
]
author_regex = re.compile("\S+\s+\(([\w ]+)\s*?\d{4,}")
commit_re = re.compile("^(\w+)\s+\(.*,\s+(\d{4,}-\d{2,}-\d{2,})\)$")
excluded_exts = set()
repo_counts: my_types.AllRepoFileAuthors = {}

def filter_ext(filename):
    ext = path.splitext(filename)[1]
    ext_whitelist = (
        ".go", ".proto", ".c", ".h", ".sh", ".md", ".xml", ".wixproj", ".wsx", ".cs"
    )
    exclude = filename == "" or ext not in ext_whitelist
    
    if exclude:
        excluded_exts.add(ext)
    
    return exclude

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    results = {}
    for name in repo_names:
        repo_root = path.join(root_dir, name)
        results[name] = count.repo(executor, repo_root, filter_func=filter_ext)

    for name, future in results.items():
        repo_counts[name] = {k: v.result() for k, v in future.items()}

end = time.perf_counter()
print(str.format("done: {}s!", round(end - start, 2)))
print(str.format("excluded file extensions: {}", excluded_exts))

In [ ]:
import git
from typing import List, Tuple, Dict

max_samples = 128
samples: List[Tuple[date, str]] = []
commits: Dict[str, date] = {}
log_lines = git.log(path.join(root_dir, repo_names[2]), "master")
for line in log_lines:
    if line == "":
        continue

    match = commit_re.match(line)
    if match is None:
        continue

    commit_hash, commit_date = [match[k] for k in (range(1, 3))]

    if commit_hash is None or commit_date is None:
        print("no commit hash or date")
        continue

    commits[commit_hash] = date.fromisoformat(commit_date)

date_hashes: List[Tuple[date, str]] = [(d, h) for h, d in commits.items()]
date_hashes.sort(key=lambda n: n[0], reverse=True)

date_hashes_len = len(date_hashes)
if max_samples > date_hashes_len:
    max_samples = date_hashes_len

interval = floor(len(date_hashes) / max_samples)

for i, date_hash in enumerate(date_hashes):
    if i % interval == 0:
        samples.append(date_hash)

max_worktrees = 10

In [ ]:
# from time import sleep
#
# import worktree_queue
# reload(worktree_queue)
#
# with WorktreeQueue("/home/bwhite/Projects/bryanchriswhite/uplink", 10) as q:
#     for 
#
#     print(q.get_root())
#     sleep(2)


In [ ]:
# def add_worktree(i: int):
#     work_path = path.join(f'worktree{i}')
#     args = ["git", "worktree", "add", work_path, "master"]
#     subprocess.run(args,
#                    capture_output=True,
#                    cwd=root_dir)
# 
# for i in range(0, max_worktrees):
#     add_worktree(i)
# 
# for i in range(0, max_worktrees):
#     remove_worktree(i)

# for c_date, hash in samples:
    #git worktree move


In [ ]:
import print, util
reload(print)

totals_dict, total_lines = util.repo_counts_to_totals_dict(repo_counts)
print.leaderboard(totals_dict)


In [ ]:
import graph
reload(graph)

graph.repo_stackplots(totals_dict)

In [ ]:
# from github import Github
#
# apikey = open("github.apikey")
#
# g = Github()
# org = g.get_organization("storj")
# repos = org.get_repos()
# page = repos.get_page(0)
#
# storj_repos = page

In [ ]:
# [(r.name, r.created_at) for r in storj_repos]